In [23]:
# import libraries
import pandas as pd

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [24]:
# download nltk corpus
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [25]:
# Load dataset
df = pd.read_csv('/content/autoinsight_reviews.csv')
df

,review,sentiment
0,"Basic insights are fine, but more customizatio...",neutral
1,Love how it highlights trends I wouldnot have ...,positive
2,Highly recommend AutoInsight for anyone dealin...,positive
3,I love how it quickly analyzes my CSV files an...,positive
4,It@s okay for basic insights but not a full BI...,neutral
...,...,...
495,Setup was seamless and I got results in seconds.,positive
496,"AutoInsight works as expected, nothing more, n...",neutral
497,"AutoInsight works as expected, nothing more, n...",neutral
498,Some results were inaccurate and misleading.,negative


In [26]:
df['review'] = df['review'].str.lower()
df

,review,sentiment
0,"basic insights are fine, but more customizatio...",neutral
1,love how it highlights trends i wouldnot have ...,positive
2,highly recommend autoinsight for anyone dealin...,positive
3,i love how it quickly analyzes my csv files an...,positive
4,it@s okay for basic insights but not a full bi...,neutral
...,...,...
495,setup was seamless and i got results in seconds.,positive
496,"autoinsight works as expected, nothing more, n...",neutral
497,"autoinsight works as expected, nothing more, n...",neutral
498,some results were inaccurate and misleading.,negative


In [27]:
import string
df['review'] = df['review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['review'].head(30)

,review
0,basic insights are fine but more customization...
1,love how it highlights trends i wouldnot have ...
2,highly recommend autoinsight for anyone dealin...
3,i love how it quickly analyzes my csv files an...
4,its okay for basic insights but not a full bi ...
5,i expected more advanced analytics but its ver...
6,sometimes i use it when i dont have time for d...
7,autoinsight kept crashing when i uploaded larg...
8,highly recommend autoinsight for anyone dealin...
9,autoinsight works as expected nothing more not...


In [28]:
df['review'] = df['review'].apply(lambda x: word_tokenize(x))
df['review']

,review
0,"[basic, insights, are, fine, but, more, custom..."
1,"[love, how, it, highlights, trends, i, wouldno..."
2,"[highly, recommend, autoinsight, for, anyone, ..."
3,"[i, love, how, it, quickly, analyzes, my, csv,..."
4,"[its, okay, for, basic, insights, but, not, a,..."
...,...
495,"[setup, was, seamless, and, i, got, results, i..."
496,"[autoinsight, works, as, expected, nothing, mo..."
497,"[autoinsight, works, as, expected, nothing, mo..."
498,"[some, results, were, inaccurate, and, mislead..."


In [29]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
# Remove 'not' from the stopwords set
stop_words.discard('not')
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
df['review']

,review
0,basic insights fine customization would help
1,love highlights trends wouldnot noticed
2,highly recommend autoinsight anyone dealing da...
3,love quickly analyzes csv files gives meaningf...
4,okay basic insights not full bi tool
...,...
495,setup seamless got results seconds
496,autoinsight works expected nothing nothing less
497,autoinsight works expected nothing nothing less
498,results inaccurate misleading


In [30]:

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download WordNet and the POS tagger if you haven't already
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert POS tags from Treebank to WordNet tags
# treebank_tag is a POS tag
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

# Function to lemmatize and apply POS tagging
def lemmatize_with_pos(text):
    tokens = word_tokenize(text) #perform tokenization
    pos_tags = nltk.pos_tag(tokens) #assigns POS tags to each word.
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for word, pos_tag in pos_tags]
    return ' '.join(lemmatized_words)

# Applying lemmatization with POS tagging to 'Review' column in dataframe
df['review'] = df['review'].apply(lemmatize_with_pos)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [31]:
# Apply Feature Extraction (Bag of Words)
vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 2803 stored elements and shape (500, 129)>

In [32]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Train a classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [34]:
# Make predictions
y_pred = classifier.predict(X_test)

In [35]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

    negative       1.00      1.00      1.00        19
     neutral       1.00      1.00      1.00        34
    positive       1.00      1.00      1.00        47

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

